# Customer Conversion Prediction

### 📖 Objective : Predict whether a customer will convert or not with Machine Learning

### Libraries Import

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="white", rc=custom_params,palette='Set1')
# Configuring styles
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import imblearn
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Load The Data

In [114]:
df = pd.read_csv('./online_shoppers_intention.csv')

In [115]:
df.head()

Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0               1                 0.000000   
1                     0.0               2                64.000000   
2                     0.0               1                 0.000000   
3                     0.0               2                 2.666667   
4                     0.0              10               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb                 2   
2         0.20       0.20         0.0         0.0   Feb                 4   
3         0.05       0.14         0.0         0.0   Feb                 3   
4         0.02       0.05         0.0         0.0   Feb                 3   

   Browser  Region  TrafficType        VisitorType  Weekend  Revenue  
0        1       1            1  Returning_Visitor    False    False  
1        2       1            2  Returning_Visitor    False    False  
2        1       9            3  Returning_Visitor    False    False  
3        2       2            4  Returning_Visitor    False    False  
4        3       1            4  Returning_Visitor     True    False

### Exploratory Data Analysis

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

### Shape of dataset

In [117]:
df.shape

(12330, 18)

### Columns Data type

In [118]:
df.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

### Columns in dataset

In [119]:
df.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

### Null values in dataset

In [120]:
df.isna().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

### Data Preprocessing

In [121]:
#Input and Target Column
input_features = df.columns[:-1]
target_features = df.columns[-1]
# Input and Target Values
X = df[input_features]
y = df[target_features]
# Numerical and Categorical Features
num_features = X.select_dtypes(include=np.number).columns.tolist()
cat_features = X.select_dtypes('object').columns.tolist()

In [122]:
# Encoding Variables
X['Month'] = X['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,
                    'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})
dummies = pd.get_dummies(df['VisitorType'], prefix = 'VisitorType', prefix_sep = '_')
X[dummies.columns] = dummies
X.drop('VisitorType',axis=1,inplace=True)
X['Weekend'] = X['Weekend'].replace({True:1,False:0})
y=y.replace({True:1,False:0})

C:\Users\OMJI\AppData\Local\Temp\ipykernel_95060\281864961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Month'] = X['Month'].replace({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,


In [123]:
# Check for imbalance dataset
y.value_counts()

0    10422
1     1908
Name: Revenue, dtype: int64

In [124]:
# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
X, y = undersample.fit_resample(X, y)

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=40)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=40)

In [126]:
scaler = StandardScaler()
scaler.fit(X_train[num_features])
X_train[num_features] = scaler.transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])
X_val[num_features] = scaler.transform(X_val[num_features])

## ANN Model Building

In [97]:
model =Sequential()
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [127]:
# Model Fitting
model.fit(X_train, y_train, batch_size = 200, epochs = 60,validation_data = (X_val, y_val))

Epoch 1/60
15/15 [==============================] - 0s 7ms/step - loss: 0.0919 - accuracy: 0.9650 - val_loss: 0.0983 - val_accuracy: 0.9662
Epoch 2/60
15/15 [==============================] - 0s 5ms/step - loss: 0.0920 - accuracy: 0.9674 - val_loss: 0.0976 - val_accuracy: 0.9631
Epoch 3/60
15/15 [==============================] - 0s 4ms/step - loss: 0.0916 - accuracy: 0.9661 - val_loss: 0.0978 - val_accuracy: 0.9692
Epoch 4/60
15/15 [==============================] - 0s 4ms/step - loss: 0.0912 - accuracy: 0.9654 - val_loss: 0.0975 - val_accuracy: 0.9662
Epoch 5/60
15/15 [==============================] - 0s 4ms/step - loss: 0.0910 - accuracy: 0.9657 - val_loss: 0.0980 - val_accuracy: 0.9662
Epoch 6/60
15/15 [==============================] - 0s 5ms/step - loss: 0.0908 - accuracy: 0.9668 - val_loss: 0.0970 - val_accuracy: 0.9692
Epoch 7/60
15/15 [==============================] - 0s 4ms/step - loss: 0.0910 - accuracy: 0.9654 - val_loss: 0.0973 - val_accuracy: 0.9662
Epoch 8/60
15/15 [==

### Evaluating Model on Test data

In [130]:
model.evaluate(X_test,y_test)

18/18 [==============================] - 0s 2ms/step - loss: 0.1171 - accuracy: 0.9546


[0.1170840859413147, 0.9546247720718384]

Achieved 95% Accuracy on unseen data

### Predicting for test data

In [131]:
preds = model.predict(X_test)
y_pred = np.array([1 * (x[0]>=0.5) for x in preds])

18/18 [==============================] - 0s 2ms/step


In [132]:
y_pred

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,